In [10]:
%load_ext autoreload
%autoreload 2

import re
import os
import sys
import bz2
import cPickle
import itertools
sys.path.insert(1, os.path.join(sys.path[0], '..'))

from ddlite import *
from ddlite.ddbiolib.datasets import PubMedCentralCorpus

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
def load_disease_dictionary():    
    dictfile = "data/dicts/stopwords.txt"
    stopwords = [line.strip().split("\t")[0] for line in open(dictfile).readlines()]
    dictfile = "data/dicts/umls_diseases_syndromes.bz2"
    diseases = {line.strip().split("\t")[0]:1 for line in bz2.BZ2File(dictfile, 'rb').readlines()}
    diseases = {word:1 for word in diseases if word not in stopwords}
    diseases = {word:1 for word in diseases if not word.isupper()}
    dictfile = "data/dicts/ncbi_training_diseases.txt"
    terms = [line.strip().split("\t")[0] for line in open(dictfile).readlines()]
    diseases.update({word:1 for word in terms if word not in stopwords})
    diseases = {word:1 for word in terms if not word.isupper()}
    return diseases

def load_acronym_dictionary():    
    dictfile = "data/dicts/umls_diseases_syndromes.bz2"
    diseases = {line.strip().split("\t")[0]:1 for line in bz2.BZ2File(dictfile, 'rb').readlines()}
    diseases = {word:1 for word in diseases if word.isupper()}
    dictfile = "data/dicts/ncbi_training_diseases.txt"
    terms = [line.strip().split("\t")[0] for line in open(dictfile).readlines()]
    diseases.update({word:1 for word in terms if word.isupper()})
    return diseases

In [12]:
INDIR = "data/documents/"
OUTDIR = "candidates/jason/"

infile = "{}/pmc-subset/".format(INDIR)
cache = "{}/pmc-subset-cache/".format(INDIR)

parser = SentenceParser()

corpus = PubMedCentralCorpus(infile, parser, cache_path=cache)
sentences = [corpus[uid]["sentences"] for uid in corpus.documents.keys()]
sentences = list(itertools.chain.from_iterable(sentences))
    
diseases = load_disease_dictionary()
acronyms = load_acronym_dictionary()
matcher_d = DictionaryMatch(label='D', dictionary=diseases, ignore_case=True)
matcher_a = DictionaryMatch(label='D', dictionary=acronyms, ignore_case=False)

# dump all candidates to a pickle file
M = Union(matcher_a, matcher_d)
candidates = Entities(sentences, M)

print candidates.num_candidates()
candidates.dump_candidates("{}/disease-candidates.pkl".format(OUTDIR))

4086
